# Text to LAS Conversion with PDAL

This notebook demonstrates converting text point cloud data to LAS format using PDAL pipelines.

In [ ]:
# Import required libraries
from pathlib import Path
import subprocess
from calipso_tool.converter import txt_to_las_pipeline, h4_to_las
from calipso_tool.txt_to_las import txt_to_las

## Method 1: Direct Text to LAS Conversion

If you already have a text file from the HDF5 conversion.

In [ ]:
# Define input text file
txt_file = Path("CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-21.2020-12D.txt")
variable_name = "Samples_Averaged_Polluted_Dust"  # Replace with your actual variable name

if txt_file.exists():
    print(f"Text file found: {txt_file}")
    
    # Convert to LAS
    las_file = txt_to_las_pipeline(
        input_txt=txt_file,
        variable_name=variable_name
    )
    
    print(f"\nLAS file created: {las_file}")
else:
    print(f"Text file not found: {txt_file}")
    print("Please run the HDF5 to text conversion first.")

## Method 2: Complete Pipeline (HDF4 → LAS)

Convert directly from HDF4 to LAS with all intermediate steps.

In [ ]:
# Define input HDF4 file
hdf4_file = Path("CAL_LID_L3_Tropospheric_APro_AllSky-Standard-V4-20.2018-12D.hdf")
variable_name = "var_to_grab"  # Replace with your actual variable name

if hdf4_file.exists():
    print(f"HDF4 file found: {hdf4_file}")
    
    # Run complete pipeline
    las_file, h5_file, txt_file = h4_to_las(
        input_h4=hdf4_file,
        variable_name=variable_name,
        keep_intermediates=True  # Keep intermediate files for inspection
    )
    
    print(f"\nPipeline complete!")
    print(f"- LAS file: {las_file}")
    print(f"- HDF5 file: {h5_file}")
    print(f"- Text file: {txt_file}")
else:
    print(f"HDF4 file not found: {hdf4_file}")

## Inspect LAS File

Use PDAL to get information about the created LAS file.

In [ ]:
# Check if LAS file exists and get info
las_files = list(Path(".").glob("*.las"))

if las_files:
    las_file = las_files[0]  # Use first LAS file found
    print(f"Inspecting: {las_file}\n")
    
    # Get summary info
    result = subprocess.run(
        ["pdal", "info", str(las_file), "--summary"],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("LAS File Summary:")
        print(result.stdout)
    else:
        print(f"Error getting info: {result.stderr}")
else:
    print("No LAS files found in current directory")

## Using Custom Pipeline Parameters

You can customize the conversion parameters for specific needs.

In [ ]:
# Custom conversion with specific scale factors
if txt_file.exists():
    custom_las = txt_to_las(
        input_txt=txt_file,
        output_las="custom_scaled.las",
        variable_name=variable_name,
        scale_x=1e-6,  # Higher precision for longitude
        scale_y=1e-6,  # Higher precision for latitude
        scale_z=0.001, # Millimeter precision for altitude
        srs="EPSG:4326"  # WGS84 coordinate system
    )
    
    print(f"Created custom LAS: {custom_las}")

## Batch Processing

Process multiple text files to LAS.

In [ ]:
# Find all text files and convert them
txt_files = list(Path(".").glob("*.txt"))

if txt_files:
    print(f"Found {len(txt_files)} text files to convert\n")
    
    for txt_file in txt_files:
        try:
            las_file = txt_to_las_pipeline(
                input_txt=txt_file,
                variable_name=variable_name
            )
            print(f"✓ Converted: {txt_file.name} → {las_file.name}")
        except Exception as e:
            print(f"✗ Failed: {txt_file.name} - {e}")
else:
    print("No text files found in current directory")

## Command Line Usage

You can also use the text to LAS converter from the command line:

```bash
# Basic usage
python -m calipso_tool.txt_to_las input.txt -v Extinction_Coefficient_532

# With custom output
python -m calipso_tool.txt_to_las input.txt -o output.las -v Temperature_Met

# With custom scaling
python -m calipso_tool.txt_to_las input.txt --scale-x 1e-6 --scale-y 1e-6 --scale-z 0.001

# Using specific pipeline JSON
python -m calipso_tool.txt_to_las input.txt -p custom_pipeline.json -v var_name
```

## Next Step: LAS to COPC

After creating LAS files, the next step is to convert them to Cloud-Optimized Point Cloud (COPC) format:

```bash
pdal pipeline las2copc.json --readers.las.filename=input.las --writers.copc.filename=output.copc.las
```